# LangGraph 101

[Los LLMs](https://python.langchain.com/docs/concepts/chat_models/) hacen posible integrar inteligencia en una nueva clase de aplicaciones. [LangGraph](https://langchain-ai.github.io/langgraph/) es un framework para ayudar a construir aplicaciones con LLMs. Aquí, revisaremos los conceptos básicos de LangGraph, explicaremos sus beneficios, mostraremos cómo usarlo para construir flujos de trabajo / agentes, y mostraremos cómo funciona con [LangChain](https://www.langchain.com/) / [LangSmith](https://docs.smith.langchain.com/).

![ecosystem](./img/ecosystem.png)

## Modelos de chat

[Los modelos de chat](https://python.langchain.com/docs/concepts/chat_models/) son la base de las aplicaciones LLM. Típicamente se accede a ellos a través de una interfaz de chat que toma una lista de [mensajes](https://python.langchain.com/docs/concepts/messages/) como entrada y devuelve un [mensaje](https://python.langchain.com/docs/concepts/messages/) como salida. LangChain proporciona [una interfaz estandarizada para modelos de chat](https://python.langchain.com/api_reference/langchain/chat_models/langchain.chat_models.base.init_chat_model.html), facilitando el [acceso a muchos proveedores diferentes](https://python.langchain.com/docs/integrations/chat/).

In [1]:
from dotenv import load_dotenv
load_dotenv("../.env", override=True)

True

In [2]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("openai:gpt-4.1", temperature=0)

## Ejecutando el modelo

La interfaz `init_chat_model` proporciona métodos [estandarizados](https://python.langchain.com/docs/concepts/runnables/) para usar modelos de chat, que incluyen:
- `invoke()`: Una sola entrada se transforma en una salida.
- `stream()`: Las salidas se [transmiten en streaming](https://python.langchain.com/docs/concepts/streaming/#stream-and-astream) a medida que se producen.

In [3]:
result = llm.invoke("What is an agent?")

In [4]:
result

AIMessage(content='The term **"agent"** can have different meanings depending on the context. Here are some common definitions:\n\n---\n\n### 1. **General Definition**\nAn **agent** is **someone or something that acts on behalf of another** or **causes an effect**.\n\n---\n\n### 2. **In Artificial Intelligence (AI) and Computer Science**\nAn **agent** is a **system or entity that perceives its environment through sensors and acts upon that environment through actuators** to achieve specific goals.  \n- **Example:** A robot vacuum cleaner is an agent: it senses dirt and obstacles, and moves to clean the floor.\n\n---\n\n### 3. **In Business and Law**\nAn **agent** is a **person authorized to act on behalf of another (the principal)** in business transactions.\n- **Example:** A real estate agent sells houses for clients.\n\n---\n\n### 4. **In Chemistry and Biology**\nAn **agent** is a **substance that brings about a chemical or biological effect**.\n- **Example:** A cleaning agent remove

In [11]:
type(result)

langchain_core.messages.ai.AIMessage

In [5]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage, AIMessageChunk

# uso de varios messages para invocar un chat model

system_message = SystemMessage(content="You are a helpful assistant that can add and multiply numbers")
user_message = HumanMessage(content="Suma 2 y 3")

llm.invoke([system_message, user_message])

AIMessage(content='La suma de 2 y 3 es 5.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 29, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'id': 'chatcmpl-BpKs6borY7NLYHgfplFGTBDhH6Fdu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--38e292ea-0cb3-4ab4-9486-1db6b4839c96-0', usage_metadata={'input_tokens': 29, 'output_tokens': 12, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
# obteniendo mensajes en streaming de una inferencia de un chat model

# iniciar una variable para sumar las respuestas de los chunks
response_text: AIMessage
for chunk in llm.stream([HumanMessage(content="Me puedes contar un chiste?")]):
    # hago un print de los tipos de chunks
    # print(type(chunk))
    # print(chunk)
    response_text += chunk.content
    
print(response_text)

¡Claro! Aquí va uno:

—¿Cuál es el animal más antiguo?
—La cebra, porque está en blanco y negro. 🦓😄


In [12]:
from rich.markdown import Markdown
Markdown(result.content)

The term "agent" can have different meanings depending on the context. Here are some common definitions:           

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                               1. General Definition                                               

An agent is someone or something that acts on behalf of another or causes an effect.                               

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                              2. In Artificial Intelligence (AI) and Computer Science                              

An agent is a system or entity that perceives its environment through sensors and acts upon that environment       
through actuators to achieve specific goals.                                                                       

 • Example: A robot vacuum cleaner is an agent: it senses dirt and obstacles, and moves to clean the floor.        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                              3. In Business and Law                                               

An agent is a person authorized to act on behalf of another (the principal) in business transactions.              

 • Example: A real estate agent sells houses for clients.                                                          

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                            4. In Chemistry and Biology                                            

An agent is a substance that brings about a chemical or biological effect.                                         

 • Example: A cleaning agent removes stains; an infectious agent causes disease.                                   

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                        5. In Literature and Entertainment                                         

An agent can refer to a representative (like a literary agent) or a character who acts with purpose.               

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Summary:                                                                                                           
An agent is generally an entity that acts, makes decisions, or causes change, often on behalf of someone or        
something else. The specific meaning depends on the field or context.

In [13]:
print(result.content)

The term **"agent"** can have different meanings depending on the context. Here are some common definitions:

---

### 1. **General Definition**
An **agent** is **someone or something that acts on behalf of another** or **causes an effect**.

---

### 2. **In Artificial Intelligence (AI) and Computer Science**
An **agent** is a **system or entity that perceives its environment through sensors and acts upon that environment through actuators** to achieve specific goals.  
- **Example:** A robot vacuum cleaner is an agent: it senses dirt and obstacles, and moves to clean the floor.

---

### 3. **In Business and Law**
An **agent** is a **person authorized to act on behalf of another (the principal)** in business transactions.
- **Example:** A real estate agent sells houses for clients.

---

### 4. **In Chemistry and Biology**
An **agent** is a **substance that brings about a chemical or biological effect**.
- **Example:** A cleaning agent removes stains; an infectious agent causes disea

## Herramientas

[Las herramientas](https://python.langchain.com/docs/concepts/tools/) son utilidades que pueden ser llamadas por un modelo de chat. En LangChain, crear herramientas se puede hacer usando el decorador `@tool`, que transforma funciones de Python en herramientas llamables. Automáticamente inferirá el nombre de la herramienta, descripción y argumentos esperados desde la definición de la función. También puedes usar [servidores del Protocolo de Contexto del Modelo (MCP)](https://github.com/langchain-ai/langchain-mcp-adapters) como herramientas compatibles con LangChain.

In [14]:
from langchain.tools import tool

@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}' and content: {content}"

In [15]:
type(write_email)

langchain_core.tools.structured.StructuredTool

In [16]:
write_email.args

{'to': {'title': 'To', 'type': 'string'},
 'subject': {'title': 'Subject', 'type': 'string'},
 'content': {'title': 'Content', 'type': 'string'}}

In [17]:
Markdown(write_email.description)

Write and send an email.

## Llamadas a herramientas

Las herramientas pueden ser [llamadas](https://python.langchain.com/docs/concepts/tool_calling/) por LLMs. Cuando una herramienta está vinculada al modelo, el modelo puede elegir llamar la herramienta devolviendo una salida estructurada con argumentos de la herramienta. Usamos el método `bind_tools` para aumentar un LLM con herramientas.

![tool-img](img/tool_call_detail.png)

Los proveedores a menudo tienen [parámetros como `tool_choice`](https://python.langchain.com/docs/how_to/tool_choice/) para forzar la llamada a herramientas específicas. `any` seleccionará al menos una de las herramientas.

Además, podemos [establecer `parallel_tool_calls=False`](https://python.langchain.com/docs/how_to/tool_calling_parallel/) para asegurar que el modelo solo llame una herramienta a la vez.

Podemos usar tool_choice para forzar la llamada a herramientas específicas. Por ejemplo, podemos establecer `tool_choice="any"` para seleccionar al menos una de las herramientas.

In [18]:
## create a chat model with langchain
llm2 = init_chat_model("openai:gpt-4o", temperature=0.6)

# hacemos dos funciones como herramientas de langchain para sumar y multiplicar
@tool
def add(a: int, b: int) -> int:
    """Suma dos números"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dos números"""
    return a * b

In [19]:
# hacemos un bind de las herramientas a nuestro chat model
tools = [add, multiply]
llm_with_tools = llm2.bind_tools(tools) # tool_choice="multiply" to enforce calling multiply always

In [20]:
# hacemos una conversación con el chat model
llm_with_tools.invoke("Suma 2 y 3")


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RtmuZgl1hzSlyehPTUQiL6UZ', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 74, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BpL59h0lOxieIiQAPNBr6J8SBn4kb', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--bed50386-fc63-4452-8ea0-ca2a3ee91da1-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 3}, 'id': 'call_RtmuZgl1hzSlyehPTUQiL6UZ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 74, 'output_tokens': 17, 'total_tokens': 91, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'o

In [21]:
# hacemos una conversación con el chat model y las herramientas
llm_with_tools.invoke("Multiplica 2 y 3")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_R6E6dcrTkYiwr0ybwNK1KeWr', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 74, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BpL8ijNsQ26qjREnFZeX1Ca4oD3Zp', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--630f4c25-26ae-408d-a050-988e72079bbd-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_R6E6dcrTkYiwr0ybwNK1KeWr', 'type': 'tool_call'}], usage_metadata={'input_tokens': 74, 'output_tokens': 17, 'total_tokens': 91, 'input_token_details': {'audio': 0, 'cache_rea

In [22]:
# Connect tools to a chat model
model_with_tools = llm.bind_tools([write_email], tool_choice="any", parallel_tool_calls=False)

# The model will now be able to call tools
output = model_with_tools.invoke("Draft a response to my boss (boss@company.ai) about tomorrow's meeting")

In [23]:
type(output)

langchain_core.messages.ai.AIMessage

In [24]:
output

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SWv6R2NOhsOO8ZlH8k8YsbGv', 'function': {'arguments': '{"to":"boss@company.ai","subject":"Re: Tomorrow\'s Meeting","content":"Hello,\\n\\nThank you for the reminder about tomorrow\'s meeting. I confirm my attendance and will be prepared with the necessary updates and materials.\\n\\nPlease let me know if there is anything specific you would like me to focus on or bring to the discussion.\\n\\nBest regards,\\n[Your Name]"}', 'name': 'write_email'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 67, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'id': 'chatcmpl-BpLAXdBkGr0QLGUQC50RXMfgWYCyd', 'service_

In [25]:
# Extract tool calls and execute them
args = output.tool_calls[0]['args']
args

{'to': 'boss@company.ai',
 'subject': "Re: Tomorrow's Meeting",
 'content': "Hello,\n\nThank you for the reminder about tomorrow's meeting. I confirm my attendance and will be prepared with the necessary updates and materials.\n\nPlease let me know if there is anything specific you would like me to focus on or bring to the discussion.\n\nBest regards,\n[Your Name]"}

In [26]:
# Call the tool
result = write_email.invoke(args)
Markdown(result)

Email sent to boss@company.ai with subject 'Re: Tomorrow's Meeting' and content: Hello,                            

Thank you for the reminder about tomorrow's meeting. I confirm my attendance and will be prepared with the         
necessary updates and materials.                                                                                   

Please let me know if there is anything specific you would like me to focus on or bring to the discussion.         

Best regards, [Your Name]

![basic_prompt](img/tool_call.png)

